In [1]:
import numpy as np
import pandas as pd
pd.__version__

'2.2.2'

In [153]:

url = '../../assets/AmesHousing.csv'
df = pd.read_csv(url) #, engine='pyarrow', dtype_backend='pyarrow')

### Shrinking Numbers

Goals:

* Create a function, `shrink_ints`, to automatically convert suitable integer columns to smaller integer types (`uint8`, `uint16`, `uint32`) based on their range of values.
* Apply the `shrink_ints` function to the DataFrame to reduce memory usage while maintaining data integrity.
* Create a function, `clean_housing`, that combines the data cleaning steps for string columns, clipping values in the "Garage Yr Blt" column, and shrinking integer columns.


In [154]:
for size in [np.uint8, np.uint16, np.uint32]:
    print(np.iinfo(size))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for uint16
---------------------------------------------------------------
min = 0
max = 65535
---------------------------------------------------------------

Machine parameters for uint32
---------------------------------------------------------------
min = 0
max = 4294967295
---------------------------------------------------------------



In [187]:
for size in [np.float16, np.float32, np.float64]:
    print(np.finfo(size))

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
smallest_normal = 6.10352e-05   smallest_subnormal = 5.96046e-08
---------------------------------------------------------------

Machine parameters for float32
---------------------------------------------------------------
precision =   6   resolution = 1.0000000e-06
machep =    -23   eps =        1.1920929e-07
negep =     -24   epsneg =     5.9604645e-08
minexp =   -126   tiny =       1.1754944e-38
maxexp =    128   max =        3.4028235e+38
nexp =        8   min =        -max
smallest_normal = 1.1754944e-38   smallest_subnormal = 1.4012985e-45
---------------------------------------------------------------

Machine parameters for float64
---

In [178]:
np.finfo(np.float64)

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

In [188]:

(
    df
    .select_dtypes(include=[np.float_])
    .memory_usage(deep=True)
    .sum()
)

257972

In [156]:
(
    df
    .select_dtypes(include=[np.int_])
    .memory_usage(deep=True)
    .sum()
)

656452

In [157]:
(
    df
    .select_dtypes(include=[np.int_])
    .describe()
    .T
)

,count,mean,std,min,25%,50%,75%,max
Order,2930.0,1.465500e+03,8.459625e+02,1.0,7.332500e+02,1465.5,2.197750e+03,2.930000e+03
PID,2930.0,7.144645e+08,1.887308e+08,526301100.0,5.284770e+08,535453620.0,9.071811e+08,1.007100e+09
MS SubClass,2930.0,5.738737e+01,4.263802e+01,20.0,2.000000e+01,50.0,7.000000e+01,1.900000e+02
Lot Area,2930.0,1.014792e+04,7.880018e+03,1300.0,7.440250e+03,9436.5,1.155525e+04,2.152450e+05
Overall Qual,2930.0,6.094881e+00,1.411026e+00,1.0,5.000000e+00,6.0,7.000000e+00,1.000000e+01
Overall Cond,2930.0,5.563140e+00,1.111537e+00,1.0,5.000000e+00,5.0,6.000000e+00,9.000000e+00
Year Built,2930.0,1.971356e+03,3.024536e+01,1872.0,1.954000e+03,1973.0,2.001000e+03,2.010000e+03
Year Remod/Add,2930.0,1.984267e+03,2.086029e+01,1950.0,1.965000e+03,1993.0,2.004000e+03,2.010000e+03
1st Flr SF,2930.0,1.159558e+03,3.918909e+02,334.0,8.762500e+02,1084.0,1.384000e+03,5.095000e+03
2nd Flr SF,2930.0,3.354560e+02,4.283957e+02,0.0,0.000000e+00,0.0,7.037500e+02,2.065000e+03


In [158]:
(
    df
    .select_dtypes(include=[np.float_])
    .describe()
    .T
)

,count,mean,std,min,25%,50%,75%,max
Lot Frontage,2440.0,69.224590,23.365335,21.0,58.0,68.0,80.0,313.0
Mas Vnr Area,2907.0,101.896801,179.112611,0.0,0.0,0.0,164.0,1600.0
BsmtFin SF 1,2929.0,442.629566,455.590839,0.0,0.0,370.0,734.0,5644.0
BsmtFin SF 2,2929.0,49.722431,169.168476,0.0,0.0,0.0,0.0,1526.0
Bsmt Unf SF,2929.0,559.262547,439.494153,0.0,219.0,466.0,802.0,2336.0
Total Bsmt SF,2929.0,1051.614544,440.615067,0.0,793.0,990.0,1302.0,6110.0
Bsmt Full Bath,2928.0,0.431352,0.524820,0.0,0.0,0.0,1.0,3.0
Bsmt Half Bath,2928.0,0.061134,0.245254,0.0,0.0,0.0,0.0,2.0
Garage Yr Blt,2771.0,1978.132443,25.528411,1895.0,1960.0,1979.0,2002.0,2207.0
Garage Cars,2929.0,1.766815,0.760566,0.0,1.0,2.0,2.0,5.0


In [169]:
def shrink_float(column: pd.Series):
    mapping = column.dtypes.name
    contains_nan = column.isna().any()
    max = column.max()
    min = column.min()
    if contains_nan:
        # fill na
        column = column.fillna(column.mean(), inplace=False).astype(np.float64)
        # return column.astype(mapping)
    if min >= np.finfo(np.float16).min and max <= np.finfo(np.float16).max:
        mapping = np.float16
    elif min >= np.finfo(np.float32).min and max <= np.finfo(np.float32).max:
        mapping = np.float32
    elif min >= np.finfo(np.float64).min and max <= np.finfo(np.float64).max:
        mapping = np.float64
    return column.astype(mapping)

In [170]:
def shrink_int(colum: pd.Series):
    mapping = colum.dtypes.name
    max_ = colum.max()
    min_ = colum.min()
    if min_ >= np.iinfo(np.uint8).min and max_ <= np.iinfo(np.uint8).max:
        mapping = np.uint8
    elif min_ >= np.iinfo(np.int16).min and max_ <= np.iinfo(np.int16).max:
        mapping = np.int16
    elif min_ >= np.iinfo(np.int32).min and max_ <= np.iinfo(np.int32).max:
        mapping = np.int32
    elif min_ >= np.iinfo(np.int64).min and max_ <= np.iinfo(np.int64).max:
        mapping = np.int64
    return colum.astype(mapping)


In [161]:
# df.select_dtypes(include=[np.int_]).apply(lambda col: shrink_int(col)).dtypes

In [162]:
(
    df
    .select_dtypes(include=[np.int_])
    .apply(lambda col: shrink_int(col))
    # .pipe(shrink_int)
    .memory_usage(deep=True)
    .sum()
)

152492

In [174]:
(152492/656452)*100

23.229725859621116

In [189]:
(
    df
    .select_dtypes(include=[np.float_])
    .apply(lambda col: shrink_float(col))
    # .pipe(shrink_int)
    .memory_usage(deep=True)
    .sum()
)

64592

In [190]:
(64592/257972)*100

25.03837625788845